In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/processed/enoe.csv")
df.head()

/var/folders/sk/b2t_fqv97kg_pjsv5f0g44nh0000gn/T/ipykernel_36398/3150187888.py:1: DtypeWarning: Columns (12,16,17,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/enoe.csv")


,Unnamed: 0,r_def,cd_a,ent,con,upm,h_mud,eda,p1a1,p1a2,...,p3m9,p3o,p3p1,p3q,p3r,p4a,p4b,encuesta,quarter,year
0,0,0,1,9,501,15769,0,35,,,...,9,2,,11.0,3,5510.0,3,1,1,10
1,1,0,1,9,501,15769,0,33,,,...,,,,NaN,,NaN,,1,1,10
2,2,0,1,9,501,15769,0,32,,,...,,2,,11.0,3,2210.0,3,1,1,10
3,3,0,1,9,501,15769,0,29,,,...,,2,,7.0,3,6112.0,2,1,1,10
4,4,0,1,9,501,15769,0,54,,,...,,,,NaN,,NaN,,1,1,10


In [5]:
pd.crosstab(index=df['p3o'], columns='count')

col_0,count
p3o,
,1935561
1,28418
2,1143564


In [6]:
pd.crosstab(index=df['p3p1'], columns='count')

col_0,count
p3p1,
,3079125
1,13225
2,14481
3,674
9,38
